In [35]:
%matplotlib inline  
# !pip install matplotlib

import pandas as pd
import numpy as np
import pandas_highcharts.core
from sqlalchemy import create_engine
import psycopg2
import matplotlib.pyplot 



import os


os.environ.setdefault("DJANGO_SETTINGS_MODULE", "IMAM.settings")

from django.conf import settings
from home.management.commands.load_data import rename_cols, assign_state_lga_num

import django
django.setup()

from home.models import Site

In [43]:
engine = create_engine(
    'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{NAME}'.format(**settings.DATABASES['default']))
df = pd.read_sql_query("select * from program;", con=engine)

In [44]:
df.tout.unique()

array([None, u'0', u'1', u'2', u'3', u'6', u'9', u'496', u'25', u'26',
       u'28', u'12', u'8', u'46', u'4', u'18', u'5', u'31', u'29', u'15',
       u'7', u'10', u'14', u'16', u'20', u'11', u'19', u'24', u'75',
       u'Mistake', u'13', u'23', u'786', u'17', u'43', u'42', u'30', u'41',
       u'52', u'36', u'232', u'48', u'108', u'114', u'59', u'35', u'33',
       u'39', u'21', u'C', u'140', u'45', u'208', u'280', u'47', u'34',
       u'56', u'133', u'SAW', u'132', u'326', u'54', u'OTP', u'S TO',
       u'0.16', u'51', u'49', u'50', u'37', u'187', u'128', u'88', u'172',
       u'201', u'275', u'325', u'55', u'68', u'63', u'number of', u'32',
       u'Pro', u'195', u'81', u'SMA', u'22', u'84', u'cancel', u'PRO',
       u'No', u'44', u'40'], dtype=object)

In [73]:
# Convert to number / float
for i in ('dcur', 'dead', 'defu', 'dmed', 'tout', 'weeknum'):
    df[i] = pd.to_numeric(df[i], errors='coerce')
    # print df_filtered[i].type()

In [74]:
# remove crap data
df_filtered = df.query('weeknum==weeknum').query('0.99<weeknum<53')
# removes none but not NaN

In [75]:
df_filtered.weeknum.unique()
# All weeknum should be int
# there should be no weeknum with decimal points

array([ 42. ,  26. ,  23. ,  22. ,  24. ,  25. ,  27. ,  28. ,  29. ,
        30. ,  31. ,  32. ,  33. ,  34. ,  35. ,  36. ,  37. ,  38. ,
        39. ,  40. ,  41. ,  43. ,  44. ,  45. ,  46. ,  47. ,  48. ,
        49. ,  50. ,  51. ,  52. ,   1. ,   2. ,   3. ,   4. ,   5. ,
         6. ,  14. ,  18. ,  11. ,  21. ,  20. ,  19. ,   7. ,  15. ,
        10. ,   1.5,   9. ,   8. ,  16. ,  12. ,  13. ,   5.1,  52.1,  17. ])

In [76]:
df_filtered.rem = df_filtered.weeknum - np.floor(df_filtered.weeknum)
df_filtered.rem.unique()
# show data with strange weeknum

array([ 0. ,  0.5,  0.1,  0.1])

In [81]:
# df_strange_wn = df_filtered.query('0<rem<1'), inplace = inplace
df_strange_wn = df_filtered[df_filtered.rem != 0]

In [82]:
df_strange_wn
# only 4 cases of weeknums with decimal points.  
# 3/4 of these errors will be deleted as not in past 8 weeks

,index,contact_uuid,urn,name,groups,siteid,first_seen,last_seen,weeknum,role,...,beg,amar,tin,dcur,dead,defu,dmed,tout,confirm,unique
3433,3423,e542626a-ee16-475c-9b23-710196fcec79,2348024753729,Robert Nokia.,"No_Email, No_Post, No_SiteID, Nut Personnel",1,2016-09-08 16:00:34,2016-09-08 16:08:33,1.5,Site,...,,None,None,NaN,NaN,NaN,NaN,NaN,None,2348024753729 2016-09-08 16:00:34
18197,18145,e3bf7c70-b376-4aca-9e39-f53316f5f993,2348024176905,Ishaku Ahmed .,"Imam Supervision, Imam Supervision 2, Imam Sup...",215110046,2017-01-09 14:56:41,2017-01-09 15:01:21,5.1,Site,...,16,0,0,0.0,0.0,0.0,0.0,0.0,Yes,2348024176905 2017-01-09 14:56:41
18607,18553,6abc34f5-6770-4c1e-aa66-391022775040,2347086850006,Musa I Aikau.,"Imam Supervision, Imam Supervision 2, Imam Sup...",2027110034,2017-01-09 17:45:32,2017-01-09 17:49:24,52.1,Site,...,89,0,0,5.0,0.0,2.0,5.0,0.0,Yes,2347086850006 2017-01-09 17:45:32
20520,20459,729cdead-65f6-410a-aa27-5f02ec352cff,2347033311675,Abdullahi Zubairu.,"AlertRUTF, Imam Supervision, Imam Supervision ...",2004110034,2016-12-21 10:46:28,2016-12-21 11:01:30,5.1,Site,...,115,6,0,27.0,27.0,0.0,0.0,0.0,Yes,2347033311675 2016-12-21 10:46:28


In [95]:
# Select all rows with the index label "Arizona"
# df.loc[:'Arizona']

# df_strange_wn.loc[:'weeknum'] = df_strange_wn['weeknum'].astype('int')
# Must have equal len keys and value when setting with an iterable

# df_strange_wn.loc[:'weeknum'] = df_strange_wn.loc[:'weeknum'].astype('int')
# Cannot convert NA to integer

df_strange_wn['weeknum'] = df_strange_wn['weeknum'].astype('int')
# This does the equivalent to floor['weeknum']

/home/robert/PycharmProjects/IMAM/ve/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [88]:
df_strange_wn

,index,contact_uuid,urn,name,groups,siteid,first_seen,last_seen,weeknum,role,...,beg,amar,tin,dcur,dead,defu,dmed,tout,confirm,unique
3433,3423,e542626a-ee16-475c-9b23-710196fcec79,2348024753729,Robert Nokia.,"No_Email, No_Post, No_SiteID, Nut Personnel",1,2016-09-08 16:00:34,2016-09-08 16:08:33,1,Site,...,,None,None,NaN,NaN,NaN,NaN,NaN,None,2348024753729 2016-09-08 16:00:34
18197,18145,e3bf7c70-b376-4aca-9e39-f53316f5f993,2348024176905,Ishaku Ahmed .,"Imam Supervision, Imam Supervision 2, Imam Sup...",215110046,2017-01-09 14:56:41,2017-01-09 15:01:21,5,Site,...,16,0,0,0.0,0.0,0.0,0.0,0.0,Yes,2348024176905 2017-01-09 14:56:41
18607,18553,6abc34f5-6770-4c1e-aa66-391022775040,2347086850006,Musa I Aikau.,"Imam Supervision, Imam Supervision 2, Imam Sup...",2027110034,2017-01-09 17:45:32,2017-01-09 17:49:24,52,Site,...,89,0,0,5.0,0.0,2.0,5.0,0.0,Yes,2347086850006 2017-01-09 17:45:32
20520,20459,729cdead-65f6-410a-aa27-5f02ec352cff,2347033311675,Abdullahi Zubairu.,"AlertRUTF, Imam Supervision, Imam Supervision ...",2004110034,2016-12-21 10:46:28,2016-12-21 11:01:30,5,Site,...,115,6,0,27.0,27.0,0.0,0.0,0.0,Yes,2347033311675 2016-12-21 10:46:28


In [93]:
df_strange_wn.weeknum.unique()

array([ 1,  5, 52])

In [96]:
df_filtered['weeknum'] = df_filtered['weeknum'].astype('int')

In [97]:
df_filtered.weeknum.unique()

array([42, 26, 23, 22, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,  1,  2,  3,
        4,  5,  6, 14, 18, 11, 21, 20, 19,  7, 15, 10,  9,  8, 16, 12, 13,
       17])

In [9]:
df.tout.unique()

array([             nan,   0.00000000e+00,   1.00000000e+00,
         2.00000000e+00,   3.00000000e+00,   6.00000000e+00,
         9.00000000e+00,   4.96000000e+02,   2.50000000e+01,
         2.60000000e+01,   2.80000000e+01,   1.20000000e+01,
         8.00000000e+00,   4.60000000e+01,   4.00000000e+00,
         1.80000000e+01,   5.00000000e+00,   3.10000000e+01,
         2.90000000e+01,   1.50000000e+01,   7.00000000e+00,
         1.00000000e+01,   1.40000000e+01,   1.60000000e+01,
         2.00000000e+01,   1.10000000e+01,   1.90000000e+01,
         2.40000000e+01,   7.50000000e+01,   1.30000000e+01,
         2.30000000e+01,   7.86000000e+02,   1.70000000e+01,
         4.30000000e+01,   4.20000000e+01,   3.00000000e+01,
         4.10000000e+01,   5.20000000e+01,   3.60000000e+01,
         2.32000000e+02,   4.80000000e+01,   1.08000000e+02,
         1.14000000e+02,   5.90000000e+01,   3.50000000e+01,
         3.30000000e+01,   3.90000000e+01,   2.10000000e+01,
         1.40000000e+02,

In [98]:
# Total Discharges from program
df_filtered['total_discharges'] = df_filtered.dcur + df_filtered.dead + df_filtered.defu + df_filtered.dmed

# Total Exits from implementation site - Cout (Mike Golden term) includes the internal transfers - tout
df_filtered['cout'] = df_filtered.total_discharges + df_filtered.tout

In [100]:
df_filtered['total_discharges']

0         NaN
1         NaN
2        13.0
3        10.0
4        16.0
5        10.0
6         3.0
7         0.0
8        24.0
9        10.0
10       10.0
11        3.0
12       24.0
13       12.0
14       18.0
15        NaN
16        5.0
17        5.0
18        7.0
19        7.0
20        4.0
21        NaN
22       12.0
23       11.0
24       18.0
25       12.0
26       11.0
27       18.0
28       22.0
29       13.0
         ... 
31285     9.0
31286     4.0
31287     2.0
31288     0.0
31289     0.0
31290     5.0
31291     0.0
31292    76.0
31293    73.0
31294    10.0
31295     7.0
31296     2.0
31297     4.0
31298     2.0
31299     0.0
31300     6.0
31301     NaN
31302     7.0
31303     4.0
31304    15.0
31305    17.0
31306    22.0
31307    53.0
31308    53.0
31309     NaN
31310     7.0
31311    10.0
31312    15.0
31313    20.0
31314    18.0
Name: total_discharges, dtype: float64

In [101]:
df_filtered['cout']

0          NaN
1          NaN
2         13.0
3         10.0
4         16.0
5         10.0
6          3.0
7          0.0
8         24.0
9         10.0
10        10.0
11         3.0
12        24.0
13        12.0
14        18.0
15         NaN
16         5.0
17         5.0
18         7.0
19         7.0
20         5.0
21         NaN
22        12.0
23        12.0
24        19.0
25        12.0
26        12.0
27        19.0
28        22.0
29        13.0
         ...  
31285      9.0
31286      4.0
31287      2.0
31288      0.0
31289      0.0
31290      5.0
31291      0.0
31292    127.0
31293     73.0
31294     14.0
31295      7.0
31296      2.0
31297      4.0
31298      4.0
31299      0.0
31300      6.0
31301      NaN
31302      7.0
31303      7.0
31304     15.0
31305     17.0
31306     22.0
31307     94.0
31308     53.0
31309      NaN
31310      7.0
31311     10.0
31312     16.0
31313     20.0
31314     18.0
Name: cout, dtype: float64

In [102]:
# Present exits and discharge / cout totals
columnsTitles = ['state_num', 'lga_num', 'siteid', 'dcur', 'dead', 'defu', 'dmed', 'tout', 'total_discharges', 'cout' ]
exit_df = df_filtered.reindex(columns=columnsTitles)

In [103]:
exit_df 
# no obvious errors in the discharge and cout totals

,state_num,lga_num,siteid,dcur,dead,defu,dmed,tout,total_discharges,cout
0,NaN,NaN,443100048,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3313,12.0,0.0,1.0,0.0,0.0,13.0,13.0
3,NaN,NaN,3308110010,10.0,0.0,0.0,0.0,0.0,10.0,10.0
4,NaN,NaN,3308110010,15.0,0.0,1.0,0.0,0.0,16.0,16.0
5,NaN,NaN,3308110010,10.0,0.0,0.0,0.0,0.0,10.0,10.0
6,NaN,NaN,3308110010,2.0,0.0,1.0,0.0,0.0,3.0,3.0
7,NaN,NaN,3308110010,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,NaN,NaN,3308110010,24.0,0.0,0.0,0.0,0.0,24.0,24.0
9,NaN,NaN,3308110010,8.0,0.0,0.0,2.0,0.0,10.0,10.0


In [106]:
# This method is almost correct - should be sum of deaths by week over sum of total discharges by week
defu_rate_by_week = df_filtered['defu'].groupby(df_filtered['weeknum']).sum() / df_filtered['total_discharges'].groupby(df_filtered['weeknum']).sum()
defu_rate_by_week = list(zip(defu_rate_by_week.index, defu_rate_by_week.values.tolist()))
print defu_rate_by_week

[(1, 0.19495900544945655), (2, 0.10870947234583599), (3, 0.11020062918422183), (4, 0.0958553001729563), (5, 0.10504851638306849), (6, 0.08367768595041322), (7, 0.028901734104046242), (8, 0.06666666666666667), (9, 1.0), (10, 0.22546419098143236), (11, 0.8588957055214724), (12, 0.0), (13, nan), (14, 0.35064935064935066), (15, 0.0), (16, nan), (17, nan), (18, 0.16574585635359115), (19, 0.0), (20, 0.6451612903225806), (21, 0.0), (22, 0.052508751458576426), (23, 0.07373271889400922), (24, 0.04649390243902439), (25, 0.03636363636363636), (26, 0.08644067796610169), (27, 0.05993690851735016), (28, 0.0871756109851348), (29, 0.1128806929309863), (30, 0.08660540239487607), (31, 0.13092932169109783), (32, 0.11118231336110221), (33, 0.21872471273668878), (34, 0.11862595419847329), (35, 0.10171610410273894), (36, 0.1117446544778432), (37, 0.072737441579209), (38, 0.09711563574933454), (39, 0.08611508731749212), (40, 0.08917197452229299), (41, 0.13070647254293907), (42, 0.10656303711869286), (43, 0.1

In [107]:
defu_week = df_filtered['defu'].groupby(df_filtered['weeknum']).sum() 
discharge_week = df_filtered['total_discharges'].groupby(df_filtered['weeknum']).sum()
defu_rate_week = defu_week / discharge_week
print defu_week

weeknum
1     6308.00
2     1197.00
3     1239.00
4      920.00
5      747.00
6      405.00
7       10.00
8        7.00
9       30.00
10      85.00
11     140.00
12       0.00
13        NaN
14      27.00
15       0.00
16        NaN
17        NaN
18      30.00
19       0.00
20     100.00
21       0.00
22      90.00
23     128.00
24      61.00
25      48.00
26     204.00
27     171.00
28     346.00
29     404.00
30     311.00
31     857.10
32     694.00
33    2703.00
34     777.00
35     895.00
36    1803.00
37     579.00
38    1398.10
39    1504.00
40    1302.00
41    2238.10
42    1625.00
43    1408.00
44    2220.17
45    2603.00
46    1783.00
47    1494.10
48    1106.00
49    1709.00
50    1451.00
51    1893.00
52    1306.00
Name: defu, dtype: float64 weeknum
1     32355.520
2     11011.000
3     11243.130
4      9597.800
5      7111.000
6      4840.000
7       346.000
8       105.000
9        30.000
10      377.000
11      163.000
12       41.000
13          NaN
14       77.000
15   

In [108]:
print discharge_week

weeknum
1     32355.520
2     11011.000
3     11243.130
4      9597.800
5      7111.000
6      4840.000
7       346.000
8       105.000
9        30.000
10      377.000
11      163.000
12       41.000
13          NaN
14       77.000
15        9.000
16          NaN
17          NaN
18      181.000
19       12.000
20      155.000
21       16.000
22     1714.000
23     1736.000
24     1312.000
25     1320.000
26     2360.000
27     2853.000
28     3969.000
29     3579.000
30     3591.000
31     6546.280
32     6242.000
33    12358.000
34     6550.000
35     8799.000
36    16135.000
37     7960.137
38    14396.240
39    17465.000
40    14601.000
41    17123.100
42    15249.190
43    13413.200
44    17389.270
45    16558.000
46    17746.000
47    16242.300
48    14078.000
49    16919.000
50    16014.000
51    16978.000
52    15380.000
Name: total_discharges, dtype: float64


In [109]:
print defu_rate_week

weeknum
1     0.194959
2     0.108709
3     0.110201
4     0.095855
5     0.105049
6     0.083678
7     0.028902
8     0.066667
9     1.000000
10    0.225464
11    0.858896
12    0.000000
13         NaN
14    0.350649
15    0.000000
16         NaN
17         NaN
18    0.165746
19    0.000000
20    0.645161
21    0.000000
22    0.052509
23    0.073733
24    0.046494
25    0.036364
26    0.086441
27    0.059937
28    0.087176
29    0.112881
30    0.086605
31    0.130929
32    0.111182
33    0.218725
34    0.118626
35    0.101716
36    0.111745
37    0.072737
38    0.097116
39    0.086115
40    0.089172
41    0.130706
42    0.106563
43    0.104971
44    0.127675
45    0.157205
46    0.100473
47    0.091988
48    0.078562
49    0.101011
50    0.090608
51    0.111497
52    0.084915
dtype: float64


In [111]:
print 6308/32355.0


0.194962138773
